In [ ]:
import os
from glob import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy import stats

%matplotlib inline

### Define image paths & some other items we'll need

Note the OpenCV variables in all-caps. These are just helpful variable names for values OpenCV uses internally. This style of all-caps is commonly found in C programming for enumeration data types (constants), and is used to help make programs easier to read and maintain.

See https://docs.opencv.org/4.2.0/d4/d86/group__imgproc__filter.html for more info on enumerations related to image filtering in OpenCV.

**You'll need to fix the path to your images here**

In [ ]:
data_dir = '../t01/'
apt_ref_path = 'apt_ref.tif'
fid_ref_path = 'fidicucial_ref.tif'
img_paths = sorted(glob(os.path.join(data_dir, '*.tif')))

In [ ]:
img_paths

In [ ]:
len(img_paths)

In [ ]:
fid_ref = Image.open(fid_ref_path)
fid_ref = np.asarray(fid_ref)

In [ ]:
fid_ref.shape

In [ ]:
plt.figure(figsize=(2, 2))
_ = plt.imshow(fid_ref, cmap='gray', vmin=0, vmax=255)

In [ ]:
dig_refs = []

for i in range(10):
    dig_ref = Image.open('dig_ref_%d.tif' % i)
    dig_ref = np.asarray(dig_ref)
    dig_refs.append(dig_ref)

In [ ]:
fig, axes = plt.subplots(1, 10, sharey=True)

for i, dig_ref in enumerate(dig_refs):
    _ = axes[i].imshow(dig_ref, cmap='gray', vmin=0, vmax=255)

In [ ]:
apt_ref_mask = Image.open(apt_ref_path)
apt_ref_mask = np.asarray(apt_ref_mask)

In [ ]:
apt_ref_mask.shape

In [ ]:
apt_ref_c, _ = cv2.findContours(apt_ref_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
len(apt_ref_c)

In [ ]:
apt_ref_c = apt_ref_c[0]

In [ ]:
c_mask = np.zeros(apt_ref_mask.shape, dtype=np.uint8)
cv2.drawContours(c_mask, [apt_ref_c], 0, 255, 1)
plt.figure(figsize=(8, 8))
_ = plt.imshow(c_mask, cmap='gray', vmin=0, vmax=255)

### Load a test image

Load the first image in the list using the PIL library (the only usage of PIL we will need). Once loaded, I check the shape and min/max values to determine the number of channels in the image and the range of values (8-bit vs 16-bit)

In [ ]:
img_path = img_paths[0]
# img_path = '../t01/BF_ST_035_APT_032_20190311094613.tif'
print(os.path.basename(img_path))

In [ ]:
img = Image.open(img_path)
img = np.asarray(img)

In [ ]:
img.shape, img.max(), img.dtype

In [ ]:
img.min(), img.max()

So we see the file is a single channel grayscale image with 16-bit pixel values. I want to work with 8-bit pixel values, so we'll scale the values down. We could have simply cast the 16-bit array to 8-bit but these operations often will automatically normalize the min/max values in the data. We don't want to alter the data other than to scale it.

First, we scale the 16-bit integers to an 8-bit range, but this creates floats. The floats are then cast to uint8.

**I'm also flipping the image horizontally after the 8-bit conversion because I found the NumPy flip method altered the original 16-bit values (don't know why)**

In [ ]:
img_8b = img / (2**8 + 1)

In [ ]:
img_8b.min(), img_8b.max()

In [ ]:
img_8b = img_8b.astype(np.uint8)

In [ ]:
img_8b = cv2.flip(img_8b, 1)

In [ ]:
img_8b.min(), img_8b.max()

In [ ]:
plt.figure(figsize=(16, 16))
_ = plt.imshow(img_8b, cmap='gray', vmin=0, vmax=255)

In [ ]:
res = cv2.matchTemplate(img_8b, fid_ref, cv2.TM_CCOEFF_NORMED)

In [ ]:
res.shape

In [ ]:
res.max()

In [ ]:
plt.figure(figsize=(16, 16))
_ = plt.imshow(res > .7, cmap='gray', vmin=0, vmax=1)

In [ ]:
contours, hierarchy = cv2.findContours(
    (res > .7).astype(np.uint8),
    cv2.RETR_LIST,
    cv2.CHAIN_APPROX_SIMPLE
)

In [ ]:
len(contours)

In [ ]:
fid_h, fid_w = fid_ref.shape

In [ ]:
c_centers = []
new_img = cv2.cvtColor(img_8b, cv2.COLOR_GRAY2RGB)

for c in contours:
    c_min_rect = cv2.minAreaRect(c)
    loc = np.array(c_min_rect[0])
    loc += (np.array(fid_ref.shape) / 2.) - 1
    loc = np.round(loc).astype(np.uint)
    
    c_centers.append(loc)
    
    cv2.circle(new_img, tuple(loc), 5, (0, 255, 0), -1)

plt.figure(figsize=(16, 16))
plt.imshow(new_img, cmap='gray', vmin=0, vmax=255)
plt.show()

### Next, we tackle the image rotation

This is done by assigning fiducials to a row, then finding the slope of each row. I took the mean of the slopes and then created a transformation matrix for that rotation angle and applied it to our base 8-bit image.

In [ ]:
centers_y = [cnt[1] for cnt in c_centers]

In [ ]:
img_h = img_8b.shape[1]

fig = plt.figure(figsize=(16, 4))
plt.xlim(0, img_h)
plt.xticks(range(0, img_h, 100))
_ = plt.hist(centers_y, bins=int(np.sqrt(img_h)))

In [ ]:
# rows are separated by roughly 220px
assigned_idx = []
centers_y = np.array(centers_y)
row_dist = 20
rows = []

for i, cy in enumerate(centers_y):
    if i in assigned_idx:
        continue
    
    row_min = cy - row_dist
    row_max = cy + row_dist
    
    in_row = np.logical_and(centers_y > row_min, centers_y < row_max)
    row_membership = np.where(in_row)
    row_members = list(row_membership[0])
    print(row_members)
    
    rows.append(row_members)
    assigned_idx.extend(row_members)

In [ ]:
rows

In [ ]:
c_centers = np.array(c_centers)

In [ ]:
r_degs = []

for r in rows:
    gradient, intercept, r_value, p_value, std_err = stats.linregress(c_centers[r])
    r_deg = np.degrees(np.arctan(gradient))
    r_degs.append(r_deg)

In [ ]:
r_degs

In [ ]:
r_deg_mean = np.mean(r_degs)

In [ ]:
r_deg_mean

In [ ]:
rows, cols = img_8b.shape

rot_mat = cv2.getRotationMatrix2D((cols/2., rows/2.), r_deg_mean, 1)
img_rot = cv2.warpAffine(img_8b, rot_mat, (cols, rows))

In [ ]:
plt.figure(figsize=(16, 16))
plt.imshow(img_rot, cmap='gray', vmin=0, vmax=255)
plt.show()

#### Here I define a function to rotate a point around another point. This allows us to transform all the fiducial center locations to the rotated space.

In [ ]:
def rotate(point, origin=(0, 0), degrees=0):
    angle = np.deg2rad(-degrees)
    
    ox, oy = origin
    px, py = point

    qx = ox + np.cos(angle) * (px - ox) - np.sin(angle) * (py - oy)
    qy = oy + np.sin(angle) * (px - ox) + np.cos(angle) * (py - oy)
    
    return qx, qy

In [ ]:
rot_c = rotate(c_centers[20], origin=(cols/2., rows/2.), degrees=r_deg_mean)

In [ ]:
c_centers[20], rot_c

#### The rotate function works, so apply it to all fiducial centers. However, while we do this I calculate the bounding boxes for the regions of interest relative to the fiducials, e.g. the apartment row/col numbers. Additionally, I collect regions I wanted to use for applying the luminosity correction...but this didn't work well and the regions I selected are inside the apartment so wouldn't be ideal for later time points when they could be filled with cells :(  I have removed the uniformity correction code from here so you don't have to install my cv2-extras library.

In [ ]:
apt_ref_mask.shape

In [ ]:
new_img = cv2.cvtColor(img_rot, cv2.COLOR_GRAY2RGB)
row_text_regions = []
col_text_regions = []
uni_corr_regions = []
apt_regions = []

for c_center in c_centers:
    rot_c = rotate(c_center, origin=(cols/2., rows/2.), degrees=r_deg_mean)
    c_int_tup = tuple(np.round(rot_c).astype(np.int))
    
    if c_int_tup[0] < 150 or c_int_tup[1] < 130:
        continue
    
    # rect for non-uniformity samples
    rect_vert1 = (c_int_tup[0] - 80, c_int_tup[1] - 50)
    rect_vert2 = (c_int_tup[0] - 30, c_int_tup[1])
    
    uni_corr_regions.append(
        [
            rect_vert1,
            (c_int_tup[0] - 30, c_int_tup[1] - 50),
            rect_vert2,
            (c_int_tup[0] - 80, c_int_tup[1])
        ]
    )
    
    # rect for row number
    row_rect_vert1 = (c_int_tup[0] - 10, c_int_tup[1] - 128)
    row_rect_vert2 = (c_int_tup[0] + 41, c_int_tup[1] - 100)
    
    row_text_regions.append(
        img_rot[c_int_tup[1] - 128:c_int_tup[1] - 100, c_int_tup[0] - 10:c_int_tup[0] + 41]
    )
    
    # rect for col number
    col_rect_vert1 = (c_int_tup[0] - 148, c_int_tup[1] - 29)
    col_rect_vert2 = (c_int_tup[0] - 97, c_int_tup[1] - 1)
        
    col_text_regions.append(
        img_rot[c_int_tup[1] - 29:c_int_tup[1] - 1, c_int_tup[0] - 148:c_int_tup[0] - 97]
    )
    
    # apt region
    apt_offset_x = c_int_tup[0] - apt_ref_mask.shape[1] - 10
    apt_offset_y = c_int_tup[1] - apt_ref_mask.shape[0] + 45
    apt_c = apt_ref_c + [apt_offset_x, apt_offset_y]
    
    cv2.circle(new_img, c_int_tup, 5, (0, 255, 0), -1)
    #cv2.rectangle(new_img, rect_vert1, rect_vert2, (0, 255, 0), 1)
    cv2.rectangle(new_img, row_rect_vert1, row_rect_vert2, (0, 255, 0), 1)
    cv2.rectangle(new_img, col_rect_vert1, col_rect_vert2, (0, 255, 0), 1)
    cv2.drawContours(new_img, [apt_c], 0, (0, 255, 0), 1)

plt.figure(figsize=(16, 16))
plt.imshow(new_img[300:800, 300:800], cmap='gray', vmin=0, vmax=255)
plt.show()

plt.figure(figsize=(16, 16))
plt.imshow(new_img, cmap='gray', vmin=0, vmax=255)
plt.show()

#### How does pattern matching work with digits?

In [ ]:
res_0 = cv2.matchTemplate(img_8b, dig_refs[7], cv2.TM_CCOEFF_NORMED)

In [ ]:
plt.figure(figsize=(16, 16))
_ = plt.imshow(res_0 > .8, cmap='gray', vmin=0, vmax=1)

### Find apartment addresses using pattern matching

In [ ]:
def identify_digit(dig_region):
    # padding is crucial & needs to be about 1/2  of the template width/height
    dig_region_pad = np.pad(dig_region, 10, mode='median')
    
    scores = []
    
    for i, dig_ref in enumerate(dig_refs):
        res = cv2.matchTemplate(dig_region_pad, dig_ref, cv2.TM_CCOEFF_NORMED)
        scores.append(res.max())
    
    return np.argmax(scores)

### Important: the 3-digit row/col regions were slightly re-defined above to ensure divisibility by 3

In [ ]:
single_digits = []

for r in row_text_regions:
    r_split = np.split(r, 3, axis=1)
    
    digits = []
    
    for sub_r in r_split:
        single_digits.append(sub_r)
        
        digits.append(str(identify_digit(sub_r)))
    
    plt.figure(figsize=(4, 4))
    plt.title(''.join(digits))
    plt.imshow(r, cmap='gray', vmin=0, vmax=255)
    plt.show()

In [ ]:
for r in col_text_regions:
    r_split = np.split(r, 3, axis=1)
    
    digits = []
    
    for sub_r in r_split:        
        digits.append(str(identify_digit(sub_r)))
    
    plt.figure(figsize=(4, 4))
    plt.title(''.join(digits))
    plt.imshow(r, cmap='gray', vmin=0, vmax=255)
    plt.show()

In [ ]:
plt.figure(figsize=(4, 4))
plt.imshow(single_digits[2], cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
res = cv2.matchTemplate(single_digits[0], dig_refs[0], cv2.TM_CCOEFF_NORMED)

In [ ]:
res

In [ ]:
identify_digit(single_digits[5])